In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

In [ ]:
#START CICIDS
from google.colab import drive
drive.mount('/content/drive')
import os
from pathlib import Path
import pandas as pd
from sklearn import datasets
import numpy as np
import random

#skip = sorted(random.sample(range(n), n - s))
filename= '/content/1. Wholesale customers data.csv'

column_names = pd.read_csv(filename, nrows=1).columns

# Sample 10,000 rows from the dataset
#train = pd.read_csv(filename, skiprows=lambda i: i > 0 and random.random() > 10000/(1000000-i), names=column_names)
train=pd.read_csv(filename)
# Specify the desired sample size
desired_sample_size = 400000

# Randomly sample the dataset to achieve the desired sample size
if len(train) > desired_sample_size:
    train = train.sample(n=desired_sample_size, random_state=92)  # Random state for reproducibility

train

Mounted at /content/drive


,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
0,2,3,12669,9656,7561,214,2674,1338
1,2,3,7057,9810,9568,1762,3293,1776
2,2,3,6353,8808,7684,2405,3516,7844
3,1,3,13265,1196,4221,6404,507,1788
4,2,3,22615,5410,7198,3915,1777,5185
...,...,...,...,...,...,...,...,...
435,1,3,29703,12051,16027,13135,182,2204
436,1,3,39228,1431,764,4510,93,2346
437,2,3,14531,15488,30243,437,14841,1867
438,1,3,10290,1981,2232,1038,168,2125


In [ ]:
# Remove erroneous 'Label' row
train.drop(train.loc[train["Label"] == "Label"].index, inplace=True)
X = train.drop(["Label"], axis=1)
# X = X[numeric_columns]

y =  train["Label"]

print(X.info())
print(y.info())

KeyError: 'Label'

In [ ]:
columns_to_drop = [
    'Dst Port',
    'Timestamp',
    'Fwd PSH Flags',
    'Bwd PSH Flags',
    'Fwd URG Flags',
    'Bwd URG Flags',
    'Flow Byts/s',  # This field had np.inf values during training, as such was removed
    'Flow Pkts/s'  # This field had np.inf values during training, as such was removed
]
train.drop(columns=columns_to_drop, inplace=True)
train.dropna(inplace=True)
train.drop_duplicates(inplace=True)


In [ ]:
classlabels=train["Label"].unique()
for i in classlabels:
  print(i,train["Label"].value_counts()[[i]].sum())

Benign 259315
DDOS attack-HOIC 21077
Bot 7588
DoS attacks-SlowHTTPTest 41
DoS attacks-Hulk 16914
Infilteration 6670
SSH-Bruteforce 4586
FTP-BruteForce 39
DoS attacks-GoldenEye 2017
DDOS attack-LOIC-UDP 73
DoS attacks-Slowloris 526
Brute Force -Web 31
Brute Force -XSS 12
SQL Injection 4


In [ ]:
X = train.drop(["Label"], axis=1)
# X = X[numeric_columns]

y =  train["Label"]

In [ ]:
train["Protocol"].unique()
# Forcing them to be strings
train = train.astype({"Protocol": str})
train["Protocol"].unique()
train = pd.get_dummies(train, columns=['Protocol'], drop_first=True)

In [ ]:
# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, stratify=y)
# Macro-level classifier - Random Forest
macro_classifier = RandomForestClassifier()
macro_classifier.fit(X_train, y_train)



RandomForestClassifier()

In [ ]:
malware_class_labels=  [label for label in classlabels if label != 'Benign']

In [ ]:
# Predict malicious or benign flows using the macro-level classifier
macro_predictions = macro_classifier.predict(X_test)

# Micro-level classifier - Single-class SVM - dictionary
micro_classifiers = {}
micro_predictions = {}

# Train a separate single-class SVM for each malware class
for malware_class in malware_class_labels:
  #radial base function is rbf ( e^-(|(x-x')|/2(sigma)^2) )
  # nu is the estimate of number of support vectors [0,1] range default 0.1

    svm_model = OneClassSVM(kernel='poly', nu=0.1)
    # Filter instances belonging to the current malware class
    malware_instances = X_train[y_train == malware_class]
    # Train the single-class SVM on instances of the current malware class
    svm_model.fit(malware_instances)
    # Store the trained SVM model
    micro_classifiers[malware_class] = svm_model
    # Predict the instances belonging to the current malware class
    micro_predictions[malware_class] = svm_model.predict(X_test)
# Determine if the instance belongs to a known malware or a zero-day malware based on predictions


In [ ]:

final_predictions = []
for i in range(len(X_test)):
    instance_predictions = [micro_predictions[malware_class][i] for malware_class in malware_class_labels]
    if -1 in instance_predictions:
        # Instance is predicted as a zero-day malware
        final_predictions.append(1) # if zero day malware then we get 1
    else:
        # Instance is predicted as a known malware
        final_predictions.append(0) # if already known malware then it is called 0



In [ ]:
# Evaluation

from sklearn.metrics import f1_score, roc_auc_score
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

# Assuming 'final_predictions' is the variable containing the predictions

# Convert the predictions to numeric format if needed
final_predictions_numeric = np.array([int(pred) for pred in final_predictions])

# Calculate the evaluation metrics
macro_f1_score = f1_score(y_test, macro_predictions, average='weighted')
precision = precision_score(y_test, macro_predictions, average='weighted')
recall = recall_score(y_test, macro_predictions, average='weighted')
accuracy = accuracy_score(y_test , macro_predictions )
# Print the evaluation metrics
print("Macro-level F1 Score:", macro_f1_score)
print("Macro-level recall Score:", recall)
print("Macro-level precision Score:", precision)
print("Macro-level accuracy Score:", accuracy)



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Macro-level F1 Score: 0.9665223654998146
Macro-level recall Score: 0.9723761106095881
Macro-level precision Score: 0.9618146582564557
Macro-level accuracy Score: 0.9723761106095881


In [ ]:
x=final_predictions_numeric.sum()


In [ ]:
print(x)


102021


In [ ]:
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import LabelEncoder

# Initialize a LabelEncoder
label_encoder = LabelEncoder()

# Encode the labels in y_test
y_test_encoded = label_encoder.fit_transform(y_test)

# Calculate the average precision score for each class separately
average_precision_scores = []

for malware_class in malware_class_labels:
    # Convert y_test_encoded to binary format for the current class
    y_test_binary = np.where(y_test_encoded == label_encoder.transform([malware_class])[0], 0,1)

    # Calculate average precision score for the current class
    average_precision = average_precision_score(y_test_binary, final_predictions)
    average_precision_scores.append(average_precision)

# Average the scores to get the overall micro-level average precision score
micro_avg_precision_score = np.mean(average_precision_scores)

print("Micro-level Average Precision Score:", micro_avg_precision_score)


Micro-level Average Precision Score: 0.9853218387016647


In [ ]:
from sklearn.metrics import hamming_loss

# Calculate Hamming loss
hamming_loss_score = hamming_loss(y_test_binary, final_predictions_numeric)
print("Hamming Loss Score:", hamming_loss_score)


Hamming Loss Score: 0.030550672304841545


In [ ]:

# from sklearn.metrics import average_precision_score

# # Calculate the average precision score for each class separately
# average_precision_scores = []

# # Convert boolean arrays to integers (0 and 1)
# y_test_int = y_test.astype(int)

# for malware_class in malware_class_labels:
#     # Convert y_test_int to binary format
#     y_test_binary = np.where(y_test_int == malware_class, 1,0)

#     # Calculate average precision score for the current class
#     average_precision = average_precision_score(y_test_binary, final_predictions)
#     average_precision_scores.append(average_precision)

# # Average the scores to get the overall micro-level average precision score
# micro_avg_precision_score = np.mean(average_precision_scores)

# print("Micro-level Average Precision Score:", micro_avg_precision_score)


In [ ]:
# from sklearn.metrics import precision_score, recall_score, f1_score

# # Calculate precision, recall, and F1-score for each class separately
# precision_scores = []
# recall_scores = []
# f1_scores = []

# for malware_class in classlabels:
#     # Convert y_test_int and final_predictions to binary format
#     y_test_binary = np.where(y_test_int == malware_class, 1, 0)
#     final_predictions_binary = np.where(final_predictions_int == malware_class, 1, 0)

#     # Calculate precision score for the current class
#     precision = precision_score(y_test_binary, final_predictions_binary)
#     precision_scores.append(precision)

#     # Calculate recall score for the current class
#     recall = recall_score(y_test_binary, final_predictions_binary)
#     recall_scores.append(recall)

#     # Calculate F1-score for the current class
#     f1 = f1_score(y_test_binary, final_predictions_binary)
#     f1_scores.append(f1)

# # Average the scores to get the overall performance
# macro_precision_score = np.mean(precision_scores)
# macro_recall_score = np.mean(recall_scores)
# macro_f1_score = np.mean(f1_scores)

# print("Macro-level Precision Score:", macro_precision_score)
# print("Macro-level Recall Score:", macro_recall_score)
# print("Macro-level F1 Score:", macro_f1_score)


In [ ]:

# micro_auc_score = roc_auc_score(y_test, final_predictions_numeric, multi_class='ovr')  # Set the 'multi_class' parameter to 'ovr'
# print("Micro-level AUC Score:", micro_auc_score)